# NSMQ - Kwame AI Project
##### Title: A script For Cropping Videos Based on Timestamps
###### By: Ernest Samuel, Team member; Data preprocessing Team
###### Date: 05-08-2023

This code is a script for cropping videos based on timestamps. It takes two input files: an Excel file containing the timestamps and a directory containing the videos to be cropped. The script first reads the Excel file and creates a Pandas DataFrame containing the timestamps. Then, it iterates through the DataFrame and crops the corresponding videos. The cropped videos are saved in the same directory as the original videos.

### The key points of use of this code are:

- It can be used to crop videos based on any set of timestamps.
- It is easy to use and can be run from the command line.
- It is efficient and can crop videos quickly.
- Make sure to specify the path to the video you wish to crop, else, the video to be croped is to be saved in the "videos" folder which is the default path in this script

The code is written in Python and uses the moviepy library. The moviepy library is a powerful library for working with videos in Python. It allows you to read, write, edit, and process videos.

To run the code, you will need to install the *moviepy library*. You can do this by running the following command in your terminal:

                pip install moviepy

Once you have installed the moviepy library, you can run the code by saving it as a Python file and then running it from the command line. For example, if you saved the code as crop_video.py, you could run it by running the following command:

                python crop_video.py
                
This will crop the videos and save them in the same directory as the original videos.

In [7]:
# Install needed library if not already installed
# pip install moviepy

# Updated  16/08/2023

#-------------------------------------------------------------Instructions Start---------------------------------------------------------------------------------#

# Instructions on running the code:

# 1. Ensure you have the 'moviepy' library installed. You can install it using pip:
#    Uncomment the following line to install the library (remove '#' at the beginning of the line):
#    !pip install moviepy

# 2. Make sure you have an Excel file containing video timestamps. 
#    Modify the 'excel_file' variable in the 'if __name__ == "__main__":' section to point to your Excel file.

# 3. Organize your video files in a folder. 
#    Set the 'videos_path' variable in the 'if __name__ == "__main__":' section to the folder containing your video files.

# 4. Run the script. You can execute it in your terminal or IDE. Make sure you are in the correct directory.

# The script will read the timestamps from the Excel file and crop the videos based on those timestamps.

#------------------------------------------------------------Instructions End ------------------------------------------------------------------------#

# Import necessary libraries
from moviepy.video.io.VideoFileClip import VideoFileClip
from datetime import datetime
import os
import pandas as pd

# Function to convert time string to seconds
def time_to_seconds(time_str):
    """
    Convert a time string in the format "HH:MM:SS" to seconds.
    """
    h, m, s = map(int, time_str.split(':'))
    return h * 3600 + m * 60 + s

# Function to crop videos based on timestamps in the Excel file
def crop_video(excel_file, input_file="videos"):
    """
    Crop a video based on the timestamps in the specified Excel file.

    Args:
        excel_file (str): The path to the Excel file containing the timestamps.
        input_file (str): The folder path where the video files are located.
    """
    # Specify the encoding (e.g., 'utf-8', 'latin1', 'ISO-8859-1', 'cp1252', etc.)

    # Try multiple encodings
    csv_encodings = ['utf-8', 'latin1', 'ISO-8859-1']  # Add more encodings if needed

    for csv_encoding in csv_encodings:
        try:
            video_data = pd.read_csv(excel_file, encoding=csv_encoding)
            break  # If successful, exit the loop
        except UnicodeDecodeError:
            continue  # Try the next encoding if decoding fails

    for _, row in video_data.iterrows():
        if not pd.isna(row['video_name']):
            video_name = row['video_name']

        if not pd.isna(row['start_time']):
            start_time = row['start_time']
            stop_time = row['stop_time']

            start_seconds = time_to_seconds(start_time)
            stop_seconds = time_to_seconds(stop_time)

            video_file = os.path.join(input_file, video_name)
            
            try:
                video_clip = VideoFileClip(video_file)
            except IOError as e:
                print(f"Error: {e}")
                print(f"Skipping video '{video_name}' because it could not be found.")
                continue  # Continue to the next video

            cropped_video = video_clip.subclip(start_seconds, stop_seconds)

            output_video_filename = f"cropped_{start_time.replace(':', '_')}_{stop_time.replace(':', '_')}_{video_name}"
            output_audio_filename = f"audio_{start_time.replace(':', '_')}_{stop_time.replace(':', '_')}_{video_name}.wav"

            output_video_path = os.path.join(input_file, output_video_filename)
            output_audio_path = os.path.join(input_file, output_audio_filename)

            # Save the cropped video
            cropped_video.write_videofile(output_video_path, codec='libx264')

            # Save the audio as a .wav file
            cropped_video.audio.write_audiofile(output_audio_path, codec='pcm_s16le')
            cropped_video.close()  # Close the video clip to release resources

    print("All done")

# Example usage:
if __name__ == "__main__":

    # observe that this relative path here is difference from the one in .ipy script
    excel_file = "videoTimeStamp.csv"  # Replace with the path to your Excel file
    videos_path = "videos"
    crop_video(excel_file, videos_path)
